In [175]:
from typing import Optional
import plotly.express as px
from itertools import product
import numpy as np
import pandas as pd
import qiskit.providers.fake_provider as fp
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import QFT

In [114]:
def make_circuit(eval_qubit: int,
                 barrier: Optional[bool] = False,
                 measure: Optional[bool] = False) -> QuantumCircuit:
    eval_qubit = eval_qubit
    total_qubit = eval_qubit + 1  # eval + target qubit 1개

    p = 0.34
    theta_p = 2 * np.arcsin(np.sqrt(p))  # QAA의 A를 만들기 위한 Ry 각도
    theta_for_q = 2 * theta_p  # Q의 각도

    qreg_q = QuantumRegister(total_qubit, 'q')
    creg_c = ClassicalRegister(total_qubit - 1, 'c')  # 하나 없어도 되나?
    circuit = QuantumCircuit(qreg_q, creg_c)

    circuit.h(qreg_q[0:eval_qubit])
    circuit.ry(theta_p, qreg_q[eval_qubit])
    if barrier:
        circuit.barrier()

    for i in range(eval_qubit):
        circuit.cry(theta_for_q * (2 ** (eval_qubit - 1 - i)), qreg_q[i], qreg_q[eval_qubit])
        if barrier:
            circuit.barrier()

    circuit.append(QFT(eval_qubit, inverse=True, do_swaps=False),
                   qreg_q[0:eval_qubit])
    if barrier:
        circuit.barrier()
    if measure:
        for i in range(eval_qubit):
            circuit.measure(qreg_q[i], creg_c[i])

    return circuit

In [167]:
def transpile_circuit(circuit: QuantumCircuit, type: str) -> QuantumCircuit:
    if type == 'ionq':
        transpiled_circuit = transpile(circuit,
                                       basis_gates=['rx', 'ry', 'rz', 'rxx', 'id'])
    if type == 'ibm_ideal':
        transpiled_circuit = transpile(circuit,
                                       basis_gates=['cx', 'id', 'rz', 'x', 'sx'])
    if type == 'ibm_yorktown':
        backend = fp.FakeYorktown()
        if circuit.num_qubits > backend.configuration().n_qubits:
            return QuantumCircuit(1,1)
        transpiled_circuit = transpile(circuit,
                                       basis_gates=['cx', 'id', 'rz', 'x', 'sx'],
                                       backend=backend)
    if type == 'ibm_tokyo':
        backend = fp.FakeTokyo()
        if circuit.num_qubits > backend.configuration().n_qubits:
            return QuantumCircuit(1,1)
        transpiled_circuit = transpile(circuit,
                                       basis_gates=['cx', 'id', 'rz', 'x', 'sx'],
                                       backend=backend)
    if type == 'ibm_guadalupe':
        backend = fp.FakeGuadalupe()
        if circuit.num_qubits > backend.configuration().n_qubits:
            return QuantumCircuit(1,1)
        transpiled_circuit = transpile(circuit,
                                       basis_gates=['cx', 'id', 'rz', 'x', 'sx'],
                                       backend=backend)

    return transpiled_circuit

In [162]:
circuit_setting = {"eval_qubit": [2, 3, 4,],
                   "barrier": [False],
                   "measure": [True, False],}
transpile_setting = ['ionq', 'ibm_tokyo']

In [163]:
result = pd.DataFrame(columns=['eval_qubit','barrier','measure','device','depth'])
for combo in product(*circuit_setting.values()):
    circuit = make_circuit(*combo)
    for type in transpile_setting:
        transpiled_circuit = transpile_circuit(circuit=circuit, type=type)
        depth = transpiled_circuit.depth()
        result.loc[len(result)] = [*combo, type, depth]

In [164]:
result

,eval_qubit,barrier,measure,device,depth
0,2,False,True,ionq,19
1,2,False,True,ibm_tokyo,28
2,2,False,False,ionq,18
3,2,False,False,ibm_tokyo,27
4,3,False,True,ionq,30
5,3,False,True,ibm_tokyo,42
6,3,False,False,ionq,29
7,3,False,False,ibm_tokyo,41
8,4,False,True,ionq,41
9,4,False,True,ibm_tokyo,66


In [158]:
#결과 plot하기
#transpile 최소값 해야하나?

In [171]:
circuit_setting = {"eval_qubit": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                   "barrier": [False],
                   "measure": [False],}
transpile_setting = ['ionq', 'ibm_ideal', 'ibm_yorktown', 'ibm_tokyo', 'ibm_guadalupe',]

In [172]:
result = pd.DataFrame(columns=['eval_qubit','barrier','measure','device','depth'])
for combo in product(*circuit_setting.values()):
    circuit = make_circuit(*combo)
    for type in transpile_setting:
        transpiled_circuit = transpile_circuit(circuit=circuit, type=type)
        depth = transpiled_circuit.depth()
        result.loc[len(result)] = [*combo, type, depth]

In [173]:
result

,eval_qubit,barrier,measure,device,depth
0,2,False,False,ionq,18
1,2,False,False,ibm_ideal,27
2,2,False,False,ibm_yorktown,27
3,2,False,False,ibm_tokyo,27
4,2,False,False,ibm_guadalupe,28
...,...,...,...,...,...
60,14,False,False,ionq,150
61,14,False,False,ibm_ideal,195
62,14,False,False,ibm_yorktown,0
63,14,False,False,ibm_tokyo,441


In [180]:
fig = px.line(result, x='eval_qubit', y='depth', color='device', symbol='device',
                 labels={'depth': 'Depth'}, 
                 title='Qubit vs Depth by Device',
                 width=800, height=500)

In [181]:
fig.show()

In [236]:
result = pd.DataFrame(columns=['eval_qubit','barrier','measure','device','depth'])
for combo in product(*circuit_setting.values()):
    circuit = make_circuit(*combo)
    for type in transpile_setting:
        depth_list = []
        for repetition in range(100):  ## adjust here to guarantee minima
            transpiled_circuit = transpile_circuit(circuit=circuit, type=type)
            depth_candidate = transpiled_circuit.depth()
            depth_list.append(depth_candidate)
        depth = min(depth_list)
        result.loc[len(result)] = [*combo, type, depth]

In [240]:
fig = px.line(result, x='eval_qubit', y='depth', color='device', symbol='device',
                 labels={'depth': 'Depth'}, 
                 title='Qubit vs Depth by Device')

In [241]:
fig.show()

In [242]:
fig.write_html('result.html')

In [215]:
eval_qubit = 4
[np.round(np.sin(i*np.pi/(2**eval_qubit))**2,3) for i in range(2**(eval_qubit-1)+1)]

[0.0, 0.038, 0.146, 0.309, 0.5, 0.691, 0.854, 0.962, 1.0]

In [226]:
for eval_qubit in range(4,10):
    print(f"Eval Qubit: {eval_qubit}, "
          # f"List: {[np.round(np.sin(i*np.pi/(2**eval_qubit))**2,6) for i in range(2**(eval_qubit-1)+1)]}, "
          f"Length: {len([np.round(np.sin(i*np.pi/(2**eval_qubit))**2,6) for i in range(2**(eval_qubit-1)+1)])}")

Eval Qubit: 4, Length: 9
Eval Qubit: 5, Length: 17
Eval Qubit: 6, Length: 33
Eval Qubit: 7, Length: 65
Eval Qubit: 8, Length: 129
Eval Qubit: 9, Length: 257
